In [16]:
import numpy as np
import pandas as pd
import os
from pydub import AudioSegment
from glob import glob
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
​
#Amostra os sinais de áudio
def amostrar(sinal:np.array, L:int):
​
    amostras = []
    step = int(len(sinal) / L)
​
    for i in range(0, len(sinal), step):
        if type(sinal[i]) == np.ndarray:
            amostras.append(np.mean(sinal[i]))
        else:
            amostras.append(sinal[i])
​
    if len(amostras) > L:
        while len(amostras) > L:
            amostras.remove(amostras[-1])
​
    return amostras
​
# Calcula a Transformada de Fourier do sinal e desloca o intervalo
def Fourier(sinal, L):
​
    fft_sinal = np.fft.fft(sinal, L)           # Calculando a FFT 
    fft_sinal = np.fft.fftshift(fft_sinal)     # Ajustando para intervalo de -pi a pi
​
    return fft_sinal
​
​
L = 13                     # Número de amostras para sinal amostrado
chaves = ['f_'+str(x) for x in range(L)]
frequencias = {}
for chave in chaves:
    frequencias[chave] = []
frequencias['passaro'] = []
​
​
arquivos = glob('/kaggle/input/birdclef-2023/train_audio/*/*.ogg')
​
# Vasculha os arquivos de áudio dos pássaros
for i in range(len(arquivos)):
    
    nome = str(arquivos[i])
    wav = nome[:-4] + '.wav'    
    sound = AudioSegment.from_ogg(nome)      # Lê o arquivo .ogg
    sound.export(wav, format="wav")          # Cria um .wav a partir do .ogg
    _, sinal = wavfile.read(wav)             # Salva o .wav para manipulá-lo
              
    sinal_amostrado = amostrar(sinal, L)
    sinal_transformado = Fourier(sinal_amostrado, L)
    sinal_magnitude = np.abs(sinal_transformado)      # Calcula o valor absoluto da transformada
​
    for index, valor in enumerate(sinal_magnitude):
        frequencias['f_' + str(index)].append(valor)  # Preenche as entradas da tabela de frequências
    
    frequencias['passaro'].append(nome.split('/')[5]) # Preenche a saída da tabela de frequências
    os.remove(wav)
​
#Cria dataset e o exporta como .csv para fácil manipulação
dataset = pd.DataFrame(frequencias)
dataset.to_csv('dataset.csv')
​
#Carrega o .csv
dataset = pd.read_csv('dataset.csv')
​
#Separa a tabela em entrada e saída
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
​
#Divide entrada e saída em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
​
#Instancia o modelo KNN com 5 vizinhos e distância euclidiana e o treina
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
​
# Faz previsão e depois compara com o teste
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)
​
#Gera arquivo de previsões(resposta)
submission = pd.DataFrame(y_pred)
submission.to_csv('submission', index=False)

SyntaxError: invalid non-printable character U+200B (3814062216.py, line 10)